In [14]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup

# Load CSV containing Kepler IDs
kepler_df = pd.read_csv('kep_conf_names.csv')

# Extract IDs
kepler_ids = kepler_df['kepid'].unique()[:15]

# Base URL format for Kepler data
base_url = "https://archive.stsci.edu/missions/kepler/lightcurves/{}/{}/"

# Directory to save downloaded data
save_dir = "kepler_flux_data"
os.makedirs(save_dir, exist_ok=True)


In [1]:
def get_fits_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [url + link['href'] for link in soup.find_all('a') if link['href'].endswith('.fits')]
    return links


In [16]:
# Limit number for testing first
for kid in kepler_ids[:10]:  
    kid_str = str(kid).zfill(9)
    subfolder = kid_str[:4]
    star_url = base_url.format(subfolder, kid_str)

    fits_links = get_fits_links(star_url)
    star_folder = os.path.join(save_dir, kid_str)
    os.makedirs(star_folder, exist_ok=True)

    print(f"Downloading {len(fits_links)} files for Kepler ID {kid_str}...")

    # Download .fits files
    for file_link in fits_links:
        filename = file_link.split('/')[-1]
        file_path = os.path.join(star_folder, filename)

        with requests.get(file_link, stream=True) as r:
            r.raise_for_status()
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    
    print(f"Finished Kepler ID: {kid_str}")


KeyboardInterrupt: 